This code is about presenting the steps that we did in terms of preprocessing the data, developing different approaches for  calculate the quality at the moment and then presenting a way to predict the quality in the future using a unique apporach of combining a random forest with an autoregressor 

## Initialize

Read Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Change here the path to yours

In [2]:
path_aggregated=r"YOUR_PATH\aggregated_numeric_data.csv"

In [2]:
#read csv file into df

df=pd.DataFrame() 
df=pd.read_csv("../data/aggregated_numeric_data.csv", sep=';', decimal = ".", dtype = {'production_order_code [None]': 'Int64'})


NameError: name 'pd' is not defined

Job Changes

In [4]:
df['change_flag'] = (df["product_code [None]"] != df["product_code [None]"].shift(1)).astype(int)

job_changes=df[df["change_flag"]==1].index
job_changes



Int64Index([     0,      3,      4,      5,      6,      7,      8,      9,
                10,   1052,
            ...
            102204, 102519, 103078, 103506, 105370, 105854, 106938, 109242,
            110028, 110029],
           dtype='int64', length=187)

Create df with spikes filtered

Update your path

In [5]:
#Read valid spikes

sigma_test=pd.read_csv("../data/slices_three_sigma.csv",decimal=".",sep=";")
sigma_test=sigma_test.drop("Unnamed: 0",axis=1)


feature = "Folienprofil 3-Sigma [%]"

df_over8=df[df[feature]>8]

times=sigma_test["local_time"]


#Filter df
filtered_df = df_over8[df_over8['local_time'].isin(times)]
filtered_df_ind=filtered_df.index

df_under8=df[df[feature]<8]
df_under8_ind=df_under8.index

comb_ind=df_under8_ind.union(filtered_df_ind)
baselne=df.loc[comb_ind]



In [6]:
baselne.reset_index(drop=True,inplace=True)
baselne.drop(columns=["change_flag"],inplace=True)

Calculate Correlations

In [7]:
'calculate correlation'

def calculate_correlations(df, feature):
    correlations = df.corr()[feature]
    correlations = correlations.drop(feature)  # Remove self-correlation
    return correlations


feature = "Folienprofil 3-Sigma [%]"



In [8]:
correlation=calculate_correlations(baselne,feature)
correlation_new=correlation.copy()

C:\Temp\ipykernel_16400\3892140961.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = df.corr()[feature]


Create Correlation df

In [9]:
correlations_df=pd.DataFrame(correlation_new)
correlations_df.columns=["Correlation"]
correlations_df["Feature"]=correlations_df.index
correlations_df.reset_index(inplace=True)
correlations_df=correlations_df.drop("index",axis=1)



Only keep most impactful features

In [10]:
impact = correlations_df[(correlations_df['Correlation'] > 0.25) | (correlations_df['Correlation'] < -0.25)]
impact_features=impact["Feature"].values
impact_features


#new df
impact_filtered=baselne[impact_features]
impact_filtered.drop("Anlage Stillstandsgrund [-]",axis=1,inplace=True)



C:\Temp\ipykernel_16400\2365553271.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact_filtered.drop("Anlage Stillstandsgrund [-]",axis=1,inplace=True)


Drop Quality Measures

In [11]:
'throw out qualit measures'
cleaned_impact=impact_filtered.copy()
cleaned_impact.drop(columns=["Gleitende Profiltoleranz [%]","Kennwert Profilverteilung [-]"],inplace=True)
cleaned_impact

,Blaskopf Zone 1 Temperatur (Soll) [°C],Blaskopf Zone 1 Temperatur [°C],Blaskopf Zone 10 Temperatur (Soll) [°C],Blaskopf Zone 10 Temperatur [°C],Blaskopf Zone 2 Temperatur (Soll) [°C],Blaskopf Zone 2 Temperatur [°C],Blaskopf Zone 3 Temperatur (Soll) [°C],Blaskopf Zone 3 Temperatur [°C],Blaskopf Zone 4 Temperatur (Soll) [°C],Blaskopf Zone 4 Temperatur [°C],...,Reversierzeit [min],Spezifische Energie gesamt [kWh/kg],Ultraschall Breite Regelabweichung [mm],Wickler A Kontaktantrieb Kraft [N],Wickler A Vorzug [N],Wickler A Wickelart [-],Wickler A Zentralantrieb Kraft [N],Wickler B Kontaktantrieb Kraft [N],Wickler B Wickelart [-],Wickler B Zentralantrieb Kraft [N]
0,180.0,179.0,180.0,199.0,180.0,186.0,180.0,180.0,180.0,180.0,...,30.0,0.347382,0.133301,85.0,130.0,2.0,86.0,85.0,2.0,86.0
1,180.0,179.0,180.0,199.0,180.0,186.0,180.0,180.0,180.0,180.0,...,30.0,0.347262,-1.050008,85.0,130.0,2.0,86.0,85.0,2.0,86.0
2,180.0,180.0,180.0,199.0,180.0,186.0,180.0,180.0,180.0,180.0,...,30.0,0.347519,-3.400024,85.0,130.0,2.0,86.0,85.0,2.0,86.0
3,180.0,179.0,180.0,199.0,180.0,186.0,180.0,180.0,180.0,179.0,...,30.0,0.349186,0.133341,85.0,130.0,2.0,85.0,85.0,2.0,85.0
4,180.0,179.0,180.0,199.0,180.0,186.0,180.0,180.0,180.0,179.0,...,30.0,0.349435,0.349996,85.0,130.0,2.0,85.0,85.0,2.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110469,180.0,179.0,195.0,222.0,180.0,190.0,180.0,182.0,180.0,180.0,...,30.0,0.477663,1.050008,120.0,200.0,3.0,120.0,120.0,3.0,120.0
110470,180.0,179.0,195.0,222.0,180.0,190.0,180.0,182.0,180.0,180.0,...,33.0,0.478283,0.416667,120.0,200.0,3.0,120.0,120.0,3.0,120.0
110471,180.0,179.0,195.0,222.0,180.0,190.0,180.0,182.0,180.0,180.0,...,30.0,0.478682,2.933329,120.0,200.0,3.0,120.0,120.0,3.0,120.0
110472,180.0,180.0,195.0,222.0,180.0,190.0,180.0,182.0,180.0,180.0,...,30.0,0.478242,-0.516683,120.0,200.0,3.0,120.0,120.0,3.0,120.0


# Overview Datasets

cleaned Impact = Correlation dataset

imputed_df= imputed corr dataset

imputed_df_full= full imp dataset

baselne= full dataset

In [12]:
sigma_values=baselne["Folienprofil 3-Sigma [%]"].values
feature="Folienprofil 3-Sigma [%]"

# Data Imputation

Corr Dataset

In [13]:
from sklearn.impute import SimpleImputer

# Create an imputer object with strategy='mean'
imputer = SimpleImputer(strategy='mean')

# Replace NaN values with column means
df_imputed = pd.DataFrame(imputer.fit_transform(cleaned_impact), columns=cleaned_impact.columns)

Full Dataset

In [14]:
df_full=baselne.copy()
df_full.drop(columns=["local_time"],inplace=True)
df_imputed_full = pd.DataFrame(imputer.fit_transform(df_full), columns=df_full.columns)
df_imputed_full

,product_code [None],production_order_code [None],Abzug Flachlegung [mm],Abzug Zentralführung [mm],Anlage Stillstandsgrund [-],Auftrag Länge (Soll) [m],Außenluft 1 [%],Bahngeschwindigkeit [m/min],Blaskopf Zone 1 Temperatur (Soll) [°C],Blaskopf Zone 1 Temperatur [°C],...,Wickler B Anpresskraft Steigung [%],Wickler B Anpresskraft [N],Wickler B Anzahl Nutzen [-],Wickler B Kontaktantrieb Kraft (Soll) [N],Wickler B Kontaktantrieb Kraft [N],Wickler B Wickelart [-],Wickler B Zentralantrieb Kraft (Soll) [N],Wickler B Zentralantrieb Kraft [N],Wobble Median [mm],Wärmetauscher Kühlleistung [kW]
0,94704621.0,3.180623e+09,1674.0,1654.0,0.0,9999999.0,83.0,55.700001,180.0,179.0,...,5.0,157.0,1.0,85.0,85.0,2.0,90.0,86.0,4.535008,100.661743
1,94704621.0,3.180623e+09,1674.0,1654.0,0.0,9999999.0,83.0,55.700001,180.0,179.0,...,5.0,157.0,1.0,85.0,85.0,2.0,90.0,86.0,5.532428,100.694974
2,94704621.0,3.180623e+09,1674.0,1654.0,0.0,9999999.0,83.0,55.700001,180.0,180.0,...,5.0,157.0,1.0,85.0,85.0,2.0,90.0,86.0,5.784532,100.686203
3,94704621.0,3.180623e+09,1674.0,1654.0,0.0,9999999.0,83.0,55.700001,180.0,179.0,...,5.0,158.0,1.0,85.0,85.0,2.0,90.0,85.0,4.677402,102.820302
4,94704621.0,3.180623e+09,1674.0,1654.0,0.0,9999999.0,83.0,55.700001,180.0,179.0,...,5.0,158.0,1.0,85.0,85.0,2.0,90.0,85.0,5.414635,101.137151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110469,94106158.0,5.017298e+07,1361.0,1367.0,0.0,999999.0,60.0,20.200001,180.0,179.0,...,8.0,0.0,1.0,120.0,120.0,3.0,130.0,120.0,1.987509,88.667254
110470,94106158.0,5.017298e+07,1361.0,1367.0,0.0,999999.0,60.0,20.200001,180.0,179.0,...,8.0,0.0,1.0,120.0,120.0,3.0,130.0,120.0,1.931403,89.042426
110471,94106158.0,5.017298e+07,1361.0,1367.0,0.0,999999.0,60.0,20.200001,180.0,179.0,...,8.0,0.0,1.0,120.0,120.0,3.0,130.0,120.0,1.775800,89.306883
110472,94106158.0,5.017298e+07,1361.0,1367.0,0.0,999999.0,60.0,20.200001,180.0,180.0,...,8.0,0.0,1.0,120.0,120.0,3.0,130.0,120.0,2.036364,89.757710


Clean colums of imputed_df_full

In [15]:
#Dropped columns : 'product_code [None]', 'production_order_code [None]',"Gleitende Profiltoleranz [%]","Folienprofil 3-Sigma [%]","Anlage Stillstandsgrund [-]","Kennwert Profilverteilung [-]"])

In [16]:
df_imputed_full=df_imputed_full.drop(columns=['product_code [None]', 'production_order_code [None]',"Gleitende Profiltoleranz [%]","Folienprofil 3-Sigma [%]","Anlage Stillstandsgrund [-]","Kennwert Profilverteilung [-]"])

# Testsets


Train/Test-Split df_imputed all

In [19]:
from sklearn.model_selection import train_test_split

# X: Features
# y: Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_imputed, sigma_values, test_size=0.2, random_state=42)


Define Artificial Testset


In [20]:
#Define Artificial Testset

train_set=cleaned_impact.iloc[:90000]
test_set=cleaned_impact.iloc[90000:]

X_new=train_set
y_new = baselne.loc[X_new.index][feature]

# # Drop any rows with missing values in X and y
X_train_corr = X_new.dropna()
Y_train_corr = y_new[X_new.index]

x_test_corr=test_set
y_test_corr=baselne.loc[x_test_corr.index][feature]

Imputed Datasets - Correlation


In [21]:
X_train_imputed=df_imputed.iloc[:90000]
Y_train_imputed=sigma_values[:90000]

x_test_imputed=df_imputed.iloc[90000:]
y_test_imputed=sigma_values[90000:]

Imputed Dataset Full

In [22]:
X_train_imputed_full=df_imputed_full.iloc[:90000]
Y_train_imputed_full=sigma_values[:90000]

x_test_imputed_full=df_imputed_full.iloc[90000:]
y_test_imputed_full=sigma_values[90000:]

In [ ]:
'Plot new RF'

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

ts=baselne.loc[x_test_new.index]["local_time"]
# Assuming you have the timestamps as a Series, and x and y as NumPy arrays
timestamps = pd.Series(ts)[:-1000]  # Series of timestamps
y_test = np.array(y_test_new)[:-1000]  # NumPy array of y_test values
y_prediction = np.array(y_pred)[:-1000]  # NumPy array of y_pred values

# Create a DataFrame from the data
data = pd.DataFrame({'Timestamp': timestamps, 'y_test': y_test, 'y_prediction': y_prediction})

# Convert the 'Timestamp' column to datetime format
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Sort the DataFrame by the 'Timestamp' column
data = data.sort_values('Timestamp')

# Plotting y_test and y_prediction over time
plt.plot(data['Timestamp'], data['y_test'], label='y_test')
plt.plot(data['Timestamp'], data['y_prediction'], label='y_prediction')

# Customize the plot
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('y_test and y_prediction over Time')
plt.legend()

# Rotate x-axis labels for better visibility (optional)
plt.xticks(rotation=45)

# Display the plot
plt.show()


NameError: name 'x_test_new' is not defined

In [ ]:
from sklearn.inspection import permutation_importance

result_artificial = permutation_importance(model_new, X_new, y_test_new, n_repeats=10, random_state=42)
feature_importance_artificial = result.importances_mean


Plot Feature Importance

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

In [55]:
#get timestamps

t1=baselne.local_time.iloc[j1:j1+1000]

t2=baselne.local_time.iloc[j2:j2+1000]

In [77]:
v1=sigma_values[j1:j1+1000]
v2=sigma_values[j2:j2+1000]



numpy.ndarray

Saving rf model via joblib

# Simulation Apporach

Simulate Data

In [48]:
from sklearn.ensemble import RandomForestRegressor
def simulate_data_upd(df_base,start):

    # Assuming you have the existing DataFrame stored in a df
    feature='Folienprofil 3-Sigma [%]'

    #Takes the first 5k rows of a product as input data --> adaptable
    df_sim=df_base.loc[start:(start+1000)]

    df_sim["Quality"]=baselne.loc[start:(start+1000)][feature]



    # Separate the features from the target variable (assuming the target is in the last column)
    features = df_sim.iloc[:, :-1]
    target = df_sim.iloc[:, -1]

    # Simulate the next 1,000 rows for each feature
    simulated_data = pd.DataFrame(columns=features.columns)

    for column in features.columns:
        current_feature = features[column]

        # Train a separate random forest model for each feature
        rf_feature_model = RandomForestRegressor()
        rf_feature_model.fit(current_feature[:-1].values.reshape(-1, 1), current_feature[1:].values)

        # Predict the next 1,000 rows for the feature
        predicted_feature = rf_feature_model.predict(current_feature[-1:].values.reshape(-1, 1))
        predicted_feature = np.concatenate([current_feature.values, predicted_feature[1:]])

    
        # Store the predicted feature values
        simulated_data[column] = predicted_feature[:1000]

      
    #actual data
    actual_rows=df_base.copy()[(start+1000):start+2000]
    actual_rows=actual_rows.reset_index()
    actual_rows.drop(columns="index",axis=1,inplace=True)

    return simulated_data,actual_rows

In [49]:
s2,a2=simulate_data_upd(df_imputed,j1)

C:\Temp\ipykernel_16400\1757106056.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim["Quality"]=baselne.loc[start:(start+1000)][feature]
C:\Temp\ipykernel_16400\1757106056.py:26: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  rf_feature_model.fit(current_feature[:-1].values.reshape(-1, 1), current_feature[1:].values)
C:\Temp\ipykernel_16400\1757106056.py:29: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treate

In [38]:
from sklearn.ensemble import RandomForestRegressor
def simulate_data(df_base,start):

    # Assuming you have the existing DataFrame stored in a df
    feature='Folienprofil 3-Sigma [%]'

    #Takes the first 5k rows of a product as input data --> adaptable
    df_sim=df_base.loc[start:(start+1000)]

    df_sim["Quality"]=baselne.loc[start:(start+1000)][feature]



    # Separate the features from the target variable (assuming the target is in the last column)
    features = df_sim.iloc[:, :-1]
    target = df_sim.iloc[:, -1]

    # Simulate the next 1,000 rows for each feature
    simulated_data = pd.DataFrame(columns=features.columns)

    for column in features.columns:
        current_feature = features[column]

        # Train a separate random forest model for each feature
        rf_feature_model = RandomForestRegressor()
        rf_feature_model.fit(current_feature[:-1].values.reshape(-1, 1), current_feature[1:].values)

        # Predict the next 1,000 rows for the feature
        predicted_feature = rf_feature_model.predict(current_feature[-1:].values.reshape(-1, 1))
        predicted_feature = np.concatenate([current_feature.values, predicted_feature])
    
        # Store the predicted feature values
        simulated_data[column] = predicted_feature[:1000]

      
    #actual data
    actual_rows=df_base.copy()[(start+1000):start+2000]
    actual_rows=actual_rows.reset_index()
    actual_rows.drop(columns="index",axis=1,inplace=True)

    return simulated_data,actual_rows

In [39]:
s1,a1=simulate_data(df_imputed,j1)

C:\Temp\ipykernel_16400\375246553.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim["Quality"]=baselne.loc[start:(start+1000)][feature]
C:\Temp\ipykernel_16400\375246553.py:26: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  rf_feature_model.fit(current_feature[:-1].values.reshape(-1, 1), current_feature[1:].values)
C:\Temp\ipykernel_16400\375246553.py:29: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated a

Measure Quality

In [25]:
    def calc_quality_sim(simulation,actual_data):
      column_deviation = (actual_data - simulation).abs()
      mean_deviation = column_deviation.mean()
    
      perc_deviations=(mean_deviation/actual_data)
      perc_col_means=perc_deviations.mean()
    
      return perc_col_means

In [26]:

j1=106938
j2=109242

# Plotting RF

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have the timestamps as a Series, and x and y as NumPy arrays
timestamps = pd.Series(time_art)[0:200]  # Series of timestamps
y_test = np.array(y_test)[0:200]   # NumPy array of y_test values
y_prediction = np.array(y_pred) [0:200]  # NumPy array of y_pred values

# Create a DataFrame from the data
data = pd.DataFrame({'Timestamp': timestamps, 'y_test': y_test, 'y_prediction': y_prediction})

# Convert the 'Timestamp' column to datetime format
data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Sort the DataFrame by the 'Timestamp' column
data = data.sort_values('Timestamp')

# Plotting y_test and y_prediction over time
plt.plot(data['Timestamp'], data['y_test'], label='y_test')
plt.plot(data['Timestamp'], data['y_prediction'], label='y_prediction')

# Customize the plot
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('y_test and y_prediction over Time')
plt.legend()

# Rotate x-axis labels for better visibility (optional)
plt.xticks(rotation=45)

# Display the plot
plt.show()


NameError: name 'time_art' is not defined

# Test for other option than RF

Neural Network


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers



# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(X_train_scaled.shape[1],)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(1))

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=0)

# Evaluate the model's performance
mse = model.evaluate(X_test_scaled, y_test)
print("Mean Squared Error:", mse)


In [ ]:
# Plot the predicted values vs. the actual values
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Prediction vs Actual Values')
plt.show()

CNN with Keras

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# Assuming you have a DataFrame 'X_train' with shape (478682, 94) containing the training features
# Assuming you have a Pandas Series 'y_train' with shape (478682) containing the target feature

# Convert y_train to a NumPy array
y_train_array = y_train.values

# Normalize the training features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Reshape the training features
X_train_reshaped = X_train_scaled.reshape(-1, 94, 1)

# Build the CNN model
model = keras.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(94, 1)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train_array, epochs=10, batch_size=32)


Gradient Boost Regressor

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Assuming you have a dataframe 'X_train' with shape (478682, 94) containing the training features
# Assuming you have a Pandas Series 'y_train' with shape (478682) containing the target feature

# Reshape the training features
X_train_reshaped = X_train.values.reshape(-1, 94, 1)

# Build the CNN model
model = keras.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(94, 1)))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(256, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)


In [ ]:
X_train

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# # Separate the features and the target variable
# X = df.drop(['Folienprofil 3-Sigma [%]'], axis=1)
# y = df['Folienprofil 3-Sigma [%]']

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the gradient boosting regression model
model1 = GradientBoostingRegressor()
model1.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model1.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


In [ ]:
# Plot the predicted values vs. the actual values
plt.scatter(y_test, y_pred, color='red', label='Predicted')
plt.scatter(y_test, y_test, color='blue', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Prediction vs Actual Values')
plt.legend()
plt.show()


# Feature Importance Artificial